## Import packages:

In [21]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk 
import gensim
import re
import unicodedata
import datetime
import collections
import random
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import treetaggerwrapper
import string
import os
import platform
treetaggerPath = str(os.getcwd()) + "/treetagger/" + str(platform.system()) + "/"

In [22]:
treetaggerPath

'/Users/albertomariopirovano/Documents/Programming/python/notebooks/tis_notebooks/treetagger/Darwin/'

## Load dataset:

In [2]:
trains24 = pd.read_csv("./Dataset/articoliS24O.csv", delimiter="\t")
trainRadiocor = pd.read_csv("./Dataset/articoliRadiocor.csv", delimiter='\t')

colsel_trains24 = trains24[['identificativo', 'body']]

colsel_trainRadiocor = trainRadiocor[['identificativo', 'body']]

colsel_merged = pd.concat([colsel_trains24, colsel_trainRadiocor])

colsel_merged.dropna(inplace=True)
colsel_merged = colsel_merged.reset_index()
colsel_merged = colsel_merged.drop('index', axis=1)

## Clean dataset:

In [3]:
print("SHAPE BEFORE REMOVING DUPLICATES: " + str(colsel_merged.shape)) 
colsel_merged.drop_duplicates(inplace=True) 
print("SHAPE AFTER REMOVING DUPLICATES: " + str(colsel_merged.shape))

colsel_merged['body'] = colsel_merged['body'].apply(lambda x : str(x).lower())
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(\\)+[a-z]\{[0-9]+\}','')
colsel_merged['body'] = colsel_merged['body'].str.replace('\n','')

#how to deal with times?
#colsel_merged['body'] = colsel_merged['body'].str.replace('[0-9]+\.[0-9]+','')

colsel_merged['body'] = colsel_merged['body'].str.replace(r'il( )?sole( )?24( )?ore','sole24ore')
colsel_merged['body'] = colsel_merged['body'].str.replace('(milion(i|e)|miliard(i|o)|euro)','')
#remove useless endings
colsel_merged['body'] = colsel_merged['body'].str.replace(r'\(c.fe.\)','')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'((\()?( )?sole24ore (radiocor)?( plus)?( )?(\))?.\-.*?)\-','')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(continua (da )?pagina [0-9]+)© riproduzione riservata(continua (da )?pagina [0-9]+)', '')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'((© )?riproduzione riser(vata)?).*','')

#format urls
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(www\.|http\:\/\/|https\:\/\/|\.com|\.net|\.org|\.it|@[a-zA-Z]+)','')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','')  

#format dots for pos_tagging, done in order to mess up with urls!
colsel_merged['body'] = colsel_merged['body'].str.replace(r':',' : ')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'.','. ')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'( )+',' ')
#adjusted html tags cleaning!
for _ in np.arange(2):
    colsel_merged['body'] = colsel_merged['body'].apply(lambda x: str(BeautifulSoup(x,"lxml").text))

#removing unprintable unicode
colsel_merged['body'].replace('\u200e','',regex=True,inplace=True)

SHAPE BEFORE REMOVING DUPLICATES: (10150, 2)
SHAPE AFTER REMOVING DUPLICATES: (9283, 2)


In [4]:
tmp= colsel_merged[colsel_merged['identificativo']=='FM20151009036ACmwyfCB']['body'].values
print(tmp)

[ "pietro scott jovane lascia la poltrona di amministratore delegato di rcsmedia group. al termine di una giornata fitta di incontri e partita in prima mattinata con il summit con le banche creditrici per la rinegoziazione del debito, il consiglio di amministrazione del gruppo editoriale ha comunicato l'uscita «consensuale» di scott jovane a partire dal prossimo 15 ottobre e l'assegnazione delle deleghe di jovane ad interim al presidente maurizio costa. «il cda di rcs mediagroup - si legge in una nota - ha preso atto della volontà dell'ad pietro scott jovane di lasciare il proprio incarico, ritenendo concluso un ciclo della vita aziendale e manifestando l'intenzione di intraprendere un nuovo percorso professionale».  inoltre il consiglio ha comunicato che per la risoluzione del contratto saranno corrisposte 150 mila al manager, a cui si sommano 600 mila lordi per il patto di non concorrenza. il board, iniziato alle 17. 30 in via rizzoli, nella sede centrale di rcs mediagroup, dopo la r

In [5]:
print(colsel_merged[colsel_merged['identificativo']=='FM20160827022ADDGKkAB']['body'].values)

[ 'la saga continua. nonostante che tutti sappiano come va a finire. ma il commissario peo che si occupa di policy digitale, günther oettinger, non ne è al corrente, evidentemente. la saga è quella degli editori che si sentono defraudati di un diritto a causa dei motori di ricerca - cioè di google - che propongono ai lettori qualche riga dei loro articoli insieme ai link per trovarli online. oettinger sta studiando una proposta di direttiva che dà agli editori il diritto di chiedere un compenso per quelle righe di testo. forse pensa così di apparire come l’eroe della saga, l’innovatore che risolve un problema a lungo dibattuto. invece, arriva ultimo. e dopo che questo genere di soluzione è già fallito in spagna e in germania. ma è veramente possibile che non lo sappia? e il vice presidente della commissione, andrus ansip, che conosce il digitale molto meglio di oettinger e che si occupa del mercato unico digitale, non gli ha detto nulla? in realtà, la realtà digitale va troppo veloce p

In [6]:
def clean_tag(tt):
    if(len(tt)<3):
        print(tt[0])
    if(tt[2]=='@card@' or tt[2]=='@ord@'):
        return tt[0]
    else:
        return tt[2]

In [7]:
def pos_tagger_ita(lst):
    corpus = {}
    stopw = set(stopwords.words("italian"))
    puntk = set(string.punctuation)
    rem = stopw | puntk | set('«') | set('»')
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='it', TAGDIR=treetaggerPath)
    for txt in lst:
        tags=tagger.tag_text(txt[1])
        pp_tags= treetaggerwrapper.make_tags(tags)
        tagged_txt = [clean_tag(t) for t in pp_tags if clean_tag(t) not in rem]
        corpus[txt[0]]= tagged_txt
    return corpus

corp = pos_tagger_ita(colsel_merged[['identificativo','body']].values.tolist())

In [8]:
corp['FM20151009036ACmwyfCB']

['pietro',
 'scott',
 'jovane',
 'lasciare',
 'poltrona',
 'amministratore',
 'delegare',
 'rcsmedia',
 'group',
 'termine',
 'giornata',
 'fitto',
 'incontro',
 'partita',
 'primo',
 'mattinata',
 'summit',
 'banca',
 'creditore',
 'rinegoziazione',
 'debito',
 'consiglio',
 'amministrazione',
 'gruppo',
 'editoriale',
 'avere',
 'comunicare',
 'uscita',
 'consensuale',
 'scott',
 'jovane',
 'partire',
 'prossimo',
 '15',
 'ottobre',
 'assegnazione',
 'delega',
 'jovane',
 'interim',
 'presidente',
 'maurizio',
 'costa',
 'cda',
 'rcs',
 'mediagroup',
 'leggere',
 'nota',
 'avere',
 'prendere',
 'atto',
 'volontà',
 'pietro',
 'scott',
 'jovane',
 'lasciare',
 'proprio',
 'incarico',
 'ritenere',
 'concludere',
 'ciclo',
 'vita',
 'aziendale',
 'manifestare',
 'intenzione',
 'intraprendere',
 'nuovo',
 'percorso',
 'professionale',
 'inoltre',
 'consiglio',
 'avere',
 'comunicare',
 'risoluzione',
 'contratto',
 'essere',
 'corrispondere',
 '150',
 'mila',
 'manager',
 'sommare',
 '60

tagger = treetaggerwrapper.TreeTagger(TAGLANG='it', TAGDIR=treetaggerPath)
tags=tagger.tag_text('l impermeabile Ciao, il 3 numero della lista sono un\' Dino e mi piace parlare latino. La penna è sul tavolo 8, news: . , ')
treetaggerwrapper.make_tags(tags)